In [1]:
# google colab specific - installing probcox
!pip3 install torch==1.7.0
!pip3 install pyro-ppl==1.5.1
!pip3 install probcox

In [2]:
# load R 
%load_ext rpy2.ipython

In [3]:
# install packages
%%R 
install.packages('survival')
install.packages('glmnet')
install.packages('devtools')
library(devtools)
install_github("michaelyanwang/dcalasso")

set.seed(9398)
library(glmnet)
library(survival)
library(dcalasso)


R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/survival_3.3-1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 6577371 bytes (6.3 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [4]:
# Modules
# =======================================================================================================================
import os
import sys
import time
import shutil
import subprocess
import tqdm

import numpy as np
import pandas as pd

import torch
from torch.distributions import constraints

import pyro
import pyro.distributions as dist

from pyro.infer import SVI, Trace_ELBO

import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

import probcox as pcox

dtype = torch.FloatTensor

# Set Seed
np.random.seed(8450)
torch.manual_seed(784985)

In [5]:
# Inference Setup
# =======================================================================================================================
# Custom linear predictor - Here: simple linear combination
def predictor(data):
    theta =  pyro.sample("theta", dist.StudentT(1, loc=0, scale=0.001).expand([data[1].shape[1], 1])).type(dtype)
    pred = torch.mm(data[1], theta)
    return(pred)

def evaluate(surv, X, rank, batchsize, sampling_proportion, iter_, run_suffix, predictor=predictor, seed=11):
    sampling_proportion[1] = batchsize
    eta=1 # paramter for optimization
    run = True # repeat initalization if NAN encounterd while training - gauge correct optimization settings
    while run:
        run = False
        pyro.clear_param_store()
        m = pcox.PCox(sampling_proportion=sampling_proportion, predictor=predictor)
        m.initialize(eta=eta, rank=rank, num_particles=3, seed=seed)
        loss=[0]
        for ii in tqdm.tqdm(range((iter_))):
            idx = np.unique(np.concatenate((np.random.choice(np.where(surv[:, -1]==1)[0], 1, replace=False), np.random.choice(range(surv.shape[0]), batchsize, replace=False))))[:batchsize]
            data=[surv[idx], X[idx]] # subsampled data
            loss.append(m.infer(data=data))
            # divergence check
            if loss[-1] != loss[-1]:
                eta = eta * 0.1
                run=True
                break
    g = m.return_guide()
    out = g.quantiles([0.025, 0.5, 0.975])
    return(out)

# Evaluation

## N=1000, P=100, Cor=0.1

In [6]:
%%R -o rd -o beta -o beta_hat

# generate data
P=100
n.subject = 1000; p.ti = P/2; p.tv = P/2; K = 20; n = n.subject/K; cor = 0.1;  lambda.grid =10^seq(-10,3,0.01);
beta0.ti = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta0.tv = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta = c(beta0.ti, beta0.tv)
dat.mat0 = as.data.frame(SIM.FUN.TVC(p.ti, p.tv, n.subject, cor, beta0.ti, beta0.tv))
dat.mat0[,'strat'] = dat.mat0[,dim(dat.mat0)[2]]%%(n.subject/20)
dat.mat0 = dat.mat0[,-(dim(dat.mat0)[2]-1)]
rd = dat.mat0

# dcalasso
start_time <- Sys.time()
mod = dcalasso(as.formula(paste0('Surv(t0,t1,status)~',paste(paste0('V',4:(P+3)),collapse='+'))),
               family = 'cox.ph',data = dat.mat0,
               K = 3, iter.os = 4, ncores = 2)
end_time <- Sys.time()
print(end_time - start_time)

beta_hat <- unname(mod$coefficients.pen)

Time difference of 3.648924 secs


In [7]:
surv = torch.from_numpy(np.asarray(rd.iloc[:, :3])).type(dtype)
total_obs = surv.shape[0]
total_events = torch.sum(surv[:, -1] == 1).numpy().tolist()

X = torch.from_numpy(np.asarray(rd.iloc[:, 3:-1])).type(dtype)

start_time = time.time()
pyro.clear_param_store()
out = evaluate(run_suffix='', rank=10, batchsize=512, iter_=25000, surv=surv, X=X, sampling_proportion=[total_obs, None, total_events, None])
print("--- %s seconds ---" % (time.time() - start_time))
theta_iden = np.sign(out['theta'][0].detach().numpy()) == np.sign(out['theta'][2].detach().numpy())

100%|██████████| 25000/25000 [05:06<00:00, 81.59it/s]

--- 307.4756815433502 seconds ---


In [8]:
print('R: ', (np.abs(beta[-25:]-beta_hat[-25:])).mean())
print('ProbCox: ', (np.abs(beta[-25:]-out['theta'][1].detach().numpy()[-25:, 0])).mean())
print('R - TP:', (beta_hat[-25:] != 0).sum(), ', FP: ', (beta_hat[:-25] != 0).sum())
print('ProbCox - TP:', theta_iden[-25:].sum(), ', FP: ', theta_iden[:-25].sum())

R:  0.060625296719049045
ProbCox:  0.06494965962351143
R - TP: 22 , FP:  25
ProbCox - TP: 14 , FP:  20


## N=1000, P=100, Cor=0.5

In [9]:
%%R -o rd -o beta -o beta_hat

# generate data
P=100
n.subject = 1000; p.ti = P/2; p.tv = P/2; K = 20; n = n.subject/K; cor = 0.5;  lambda.grid = 10^seq(-10,3,0.01);
beta0.ti = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta0.tv = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta = c(beta0.ti, beta0.tv)
dat.mat0 = as.data.frame(SIM.FUN.TVC(p.ti, p.tv, n.subject, cor, beta0.ti, beta0.tv))
dat.mat0[,'strat'] = dat.mat0[,dim(dat.mat0)[2]]%%(n.subject/20)
dat.mat0 = dat.mat0[,-(dim(dat.mat0)[2]-1)]
rd = dat.mat0

# dcalasso
start_time <- Sys.time()
mod = dcalasso(as.formula(paste0('Surv(t0,t1,status)~',paste(paste0('V',4:(P+3)),collapse='+'))),
               family = 'cox.ph',data = dat.mat0,
               K = 3, iter.os = 4, ncores = 2)
end_time <- Sys.time()
print(end_time - start_time)
beta_hat <- unname(mod$coefficients.pen)


Time difference of 3.950558 secs


In [10]:
surv = torch.from_numpy(np.asarray(rd.iloc[:, :3])).type(dtype)
total_obs = surv.shape[0]
total_events = torch.sum(surv[:, -1] == 1).numpy().tolist()

X = torch.from_numpy(np.asarray(rd.iloc[:, 3:-1])).type(dtype)

start_time = time.time()
pyro.clear_param_store()
out = evaluate(run_suffix='', rank=10, batchsize=512, iter_=25000, surv=surv, X=X, sampling_proportion=[total_obs, None, total_events, None])
print("--- %s seconds ---" % (time.time() - start_time))
theta_iden = np.sign(out['theta'][0].detach().numpy()) == np.sign(out['theta'][2].detach().numpy())

100%|██████████| 25000/25000 [05:15<00:00, 79.15it/s]

--- 316.4164550304413 seconds ---


In [11]:
print('R: ', (np.abs(beta[-25:]-beta_hat[-25:])).mean())
print('ProbCox: ', (np.abs(beta[-25:]-out['theta'][1].detach().numpy()[-25:, 0])).mean())
print('R - TP:', (beta_hat[-25:] != 0).sum(), ', FP: ', (beta_hat[:-25] != 0).sum())
print('ProbCox - TP:', theta_iden[-25:].sum(), ', FP: ', theta_iden[:-25].sum())

R:  0.08118207479293968
ProbCox:  0.0835316979135779
R - TP: 16 , FP:  21
ProbCox - TP: 14 , FP:  15


## N=5000, P=400, Cor=0.1

In [12]:
%%R -o rd -o beta -o beta_hat

# generate data
P=400
n.subject = 5000; p.ti = P/2; p.tv = P/2; K = 20; n = n.subject/K; cor = 0.1;  lambda.grid = 10^seq(-10,3,0.01);
beta0.ti = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta0.tv = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta = c(beta0.ti, beta0.tv)
dat.mat0 = as.data.frame(SIM.FUN.TVC(p.ti, p.tv, n.subject, cor, beta0.ti, beta0.tv))
dat.mat0[,'strat'] = dat.mat0[,dim(dat.mat0)[2]]%%(n.subject/20)
dat.mat0 = dat.mat0[,-(dim(dat.mat0)[2]-1)]
rd = dat.mat0

# dcalasso
start_time <- Sys.time()
mod = dcalasso(as.formula(paste0('Surv(t0,t1,status)~',paste(paste0('V',4:(P+3)),collapse='+'))),
               family = 'cox.ph',data = dat.mat0,
               K = 3, iter.os = 4, ncores = 2)
end_time <- Sys.time()
print(end_time - start_time)
beta_hat <- unname(mod$coefficients.pen)


Time difference of 30.0825 secs


In [13]:
surv = torch.from_numpy(np.asarray(rd.iloc[:, :3])).type(dtype)
total_obs = surv.shape[0]
total_events = torch.sum(surv[:, -1] == 1).numpy().tolist()

X = torch.from_numpy(np.asarray(rd.iloc[:, 3:-1])).type(dtype)

start_time = time.time()
pyro.clear_param_store()
out = evaluate(run_suffix='', rank=10, batchsize=512, iter_=25000, surv=surv, X=X, sampling_proportion=[total_obs, None, total_events, None])
print("--- %s seconds ---" % (time.time() - start_time))
theta_iden = np.sign(out['theta'][0].detach().numpy()) == np.sign(out['theta'][2].detach().numpy())

100%|██████████| 25000/25000 [07:43<00:00, 53.92it/s]

--- 464.11734986305237 seconds ---


In [14]:
print('R: ', (np.abs(beta[-25:]-beta_hat[-25:])).mean())
print('ProbCox: ', (np.abs(beta[-25:]-out['theta'][1].detach().numpy()[-25:, 0])).mean())
print('R - TP:', (beta_hat[-25:] != 0).sum(), ', FP: ', (beta_hat[:-25] != 0).sum())
print('ProbCox - TP:', theta_iden[-25:].sum(), ', FP: ', theta_iden[:-25].sum())

R:  0.027549150266286713
ProbCox:  0.01727140598167957
R - TP: 22 , FP:  27
ProbCox - TP: 21 , FP:  23


## N=5000, P=400, Cor=0.5


In [15]:
%%R -o rd -o beta -o beta_hat

# generate data
P=400
n.subject = 5000; p.ti = P/2; p.tv = P/2; K = 20; n = n.subject/K; cor = 0.5;  lambda.grid = 10^seq(-10,3,0.01);
beta0.ti = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta0.tv = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta = c(beta0.ti, beta0.tv)
dat.mat0 = as.data.frame(SIM.FUN.TVC(p.ti, p.tv, n.subject, cor, beta0.ti, beta0.tv))
dat.mat0[,'strat'] = dat.mat0[,dim(dat.mat0)[2]]%%(n.subject/20)
dat.mat0 = dat.mat0[,-(dim(dat.mat0)[2]-1)]
rd = dat.mat0

# dcalasso
start_time <- Sys.time()
mod = dcalasso(as.formula(paste0('Surv(t0,t1,status)~',paste(paste0('V',4:(P+3)),collapse='+'))),
               family = 'cox.ph',data = dat.mat0,
               K = 3, iter.os = 4, ncores = 2)
end_time <- Sys.time()
print(end_time - start_time)
beta_hat <- unname(mod$coefficients.pen)

Time difference of 30.88695 secs


In [16]:
surv = torch.from_numpy(np.asarray(rd.iloc[:, :3])).type(dtype)
total_obs = surv.shape[0]
total_events = torch.sum(surv[:, -1] == 1).numpy().tolist()

X = torch.from_numpy(np.asarray(rd.iloc[:, 3:-1])).type(dtype)

start_time = time.time()
pyro.clear_param_store()
out = evaluate(run_suffix='', rank=3, batchsize=1024, iter_=25000, surv=surv, X=X, sampling_proportion=[total_obs, None, total_events, None])
print("--- %s seconds ---" % (time.time() - start_time))
theta_iden = np.sign(out['theta'][0].detach().numpy()) == np.sign(out['theta'][2].detach().numpy())

100%|██████████| 25000/25000 [11:22<00:00, 36.62it/s]

--- 683.7035093307495 seconds ---


In [17]:
print('R: ', (np.abs(beta[-25:]-beta_hat[-25:])).mean())
print('ProbCox: ', (np.abs(beta[-25:]-out['theta'][1].detach().numpy()[-25:, 0])).mean())
print('R - TP:', (beta_hat[-25:] != 0).sum(), ', FP: ', (beta_hat[:-25] != 0).sum())
print('ProbCox - TP:', theta_iden[-25:].sum(), ', FP: ', theta_iden[:-25].sum())

R:  0.03279540428932915
ProbCox:  0.025973544819523835
R - TP: 21 , FP:  24
ProbCox - TP: 20 , FP:  20


## N=10000, P=800, Cor=0.1

In [18]:
%%R -o rd -o beta -o beta_hat

# generate data
P=800
n.subject = 10000; p.ti = P/2; p.tv = P/2; K = 20; n = n.subject/K; cor = 0.1;  lambda.grid = 10^seq(-10,3,0.01);
beta0.ti = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta0.tv = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta = c(beta0.ti, beta0.tv)
dat.mat0 = as.data.frame(SIM.FUN.TVC(p.ti, p.tv, n.subject, cor, beta0.ti, beta0.tv))
dat.mat0[,'strat'] = dat.mat0[,dim(dat.mat0)[2]]%%(n.subject/20)
dat.mat0 = dat.mat0[,-(dim(dat.mat0)[2]-1)]
rd = dat.mat0

# dcalasso
start_time <- Sys.time()
mod = dcalasso(as.formula(paste0('Surv(t0,t1,status)~',paste(paste0('V',4:(P+3)),collapse='+'))),
               family = 'cox.ph',data = dat.mat0,
               K = 3, iter.os = 4, ncores = 2)
end_time <- Sys.time()
print(end_time - start_time)
beta_hat <- unname(mod$coefficients.pen)


Time difference of 4.883976 mins


In [19]:
surv = torch.from_numpy(np.asarray(rd.iloc[:, :3])).type(dtype)
total_obs = surv.shape[0]
total_events = torch.sum(surv[:, -1] == 1).numpy().tolist()

X = torch.from_numpy(np.asarray(rd.iloc[:, 3:-1])).type(dtype)

start_time = time.time()
pyro.clear_param_store()
out = evaluate(run_suffix='', rank=10, batchsize=512, iter_=25000, surv=surv, X=X, sampling_proportion=[total_obs, None, total_events, None])

print("--- %s seconds ---" % (time.time() - start_time))
theta_iden = np.sign(out['theta'][0].detach().numpy()) == np.sign(out['theta'][2].detach().numpy())

100%|██████████| 25000/25000 [10:35<00:00, 39.35it/s]

--- 636.848123550415 seconds ---


In [20]:
print('R: ', (np.abs(beta[-25:]-beta_hat[-25:])).mean())
print('ProbCox: ', (np.abs(beta[-25:]-out['theta'][1].detach().numpy()[-25:, 0])).mean())
print('R - TP:', (beta_hat[-25:] != 0).sum(), ', FP: ', (beta_hat[:-25] != 0).sum())
print('ProbCox - TP:', theta_iden[-25:].sum(), ', FP: ', theta_iden[:-25].sum())

R:  0.019533126415645235
ProbCox:  0.019646703069698818
R - TP: 24 , FP:  26
ProbCox - TP: 22 , FP:  23


## N=10000, P=800, Cor=0.5


In [21]:
%%R -o rd -o beta

# generate data
P=800
n.subject = 10000; p.ti = P/2; p.tv = P/2; K = 20; n = n.subject/K; cor = 0.5;  lambda.grid = 10^seq(-10,3,0.01);
beta0.ti = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta0.tv = c(rep(0,P/2-25), rnorm(25, 0, 0.5))
beta = c(beta0.ti, beta0.tv)
dat.mat0 = as.data.frame(SIM.FUN.TVC(p.ti, p.tv, n.subject, cor, beta0.ti, beta0.tv))
dat.mat0[,'strat'] = dat.mat0[,dim(dat.mat0)[2]]%%(n.subject/20)
dat.mat0 = dat.mat0[,-(dim(dat.mat0)[2]-1)]
rd = dat.mat0

# dcalasso
start_time <- Sys.time()
mod = dcalasso(as.formula(paste0('Surv(t0,t1,status)~',paste(paste0('V',4:(P+3)),collapse='+'))),
               family = 'cox.ph',data = dat.mat0,
               K = 3, iter.os = 4, ncores = 2)
end_time <- Sys.time()
print(end_time - start_time)
beta_hat <- unname(mod$coefficients.pen)

Time difference of 4.494337 mins


In [22]:
surv = torch.from_numpy(np.asarray(rd.iloc[:, :3])).type(dtype)
total_obs = surv.shape[0]
total_events = torch.sum(surv[:, -1] == 1).numpy().tolist()

X = torch.from_numpy(np.asarray(rd.iloc[:, 3:-1])).type(dtype)

start_time = time.time()
pyro.clear_param_store()
out = evaluate(run_suffix='', rank=10, batchsize=512, iter_=25000, surv=surv, X=X, sampling_proportion=[total_obs, None, total_events, None])

print("--- %s seconds ---" % (time.time() - start_time))
theta_iden = np.sign(out['theta'][0].detach().numpy()) == np.sign(out['theta'][2].detach().numpy())

100%|██████████| 25000/25000 [10:19<00:00, 40.37it/s]

--- 620.3477699756622 seconds ---


In [23]:
print('R: ', (np.abs(beta[-25:]-beta_hat[-25:])).mean())
print('ProbCox: ', (np.abs(beta[-25:]-out['theta'][1].detach().numpy()[-25:, 0])).mean())
print('R - TP:', (beta_hat[-25:] != 0).sum(), ', FP: ', (beta_hat[:-25] != 0).sum())
print('ProbCox - TP:', theta_iden[-25:].sum(), ', FP: ', theta_iden[:-25].sum())

R:  0.019533126415645235
ProbCox:  0.02105315231633649
R - TP: 24 , FP:  26
ProbCox - TP: 20 , FP:  21
